In [ ]:
import keras
import math
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from collections import defaultdict
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# NLP
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import emoji
from nltk.corpus import stopwords

# Viz
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib as mpl

In [ ]:
# Load the dataset from CSV file
df = pd.read_csv('data_final.csv')

# Select specific columns using double square brackets
df = df[['at', 'comment', 'label']]

# Cut the last 10 characters in each cell of the 'at' column
df['at'] = df['at'].apply(lambda x: x[:-9] if isinstance(x, str) and len(x) > 10 else x)


In [ ]:
# Label encoding for the 'label' column
le = LabelEncoder()
df['label'] = le.fit_transform(df['label'])

# Filter out rows where 'comment' is not a string
df = df[df['comment'].apply(lambda x: isinstance(x, str))]

# Split data into train and test sets
train_data, test_data, train_labels, test_labels = train_test_split(
    df['comment'], df['label'], test_size=0.2, random_state=42
)

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=20000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_data)

# Sequencing and padding
train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

train_padded = pad_sequences(train_sequences, maxlen=100, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=100, padding='post', truncating='post')

In [ ]:
from keras.callbacks import ModelCheckpoint

# Definisi model
model = keras.Sequential([
    keras.layers.Embedding(20000, 50, input_length=100),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# Menyusun ModelCheckpoint callback
checkpoint = ModelCheckpoint('testing.h5',
                             monitor='val_accuracy',
                             save_best_only=True,
                             mode='max',
                             verbose=1)

# Melatih model dengan menggunakan callback ModelCheckpoint
history = model.fit(train_padded, train_labels,
                    epochs=5,
                    validation_data=(test_padded, test_labels),
                    callbacks=[checkpoint])


In [ ]:
# Plot akurasi training dan validasi
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
from wordcloud import WordCloud
nltk.download('stopwords')
from nltk.corpus import stopwords
text_data = ' '.join(df['comment'].astype(str))

wordcloud = WordCloud(width=800, height=400, background_color='white', stopwords=stopwords.words('indonesian')).generate(text_data)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud')
plt.show()